## NYC Taxi Data Experimnt Tracking

In [ ]:
!python -V

In [ ]:
import requests
import pickle

import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVR

from tqdm import tqdm

In [ ]:
files = [('green_tripdata_2022-02.parquet', './data'), ('green_tripdata_2022-01.parquet', './data')]

print("Download files:")
for file, path in files:
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    save_path=f"{path}/{file}"
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                        desc=f"{file}",
                        postfix=f"save to {save_path}",
                        total=int(resp.headers["Content-Length"])):
            handle.write(data)

In [ ]:
train_raw_data = pd.read_parquet('data/green_tripdata_2022-01.parquet')
val_raw_data = pd.read_parquet('data/green_tripdata_2022-02.parquet')

In [ ]:
train_raw_data.head()

In [ ]:
train_raw_data.info()

In [ ]:
def process_dataframe(data):
    data.lpep_dropoff_datetime = pd.to_datetime(data.lpep_dropoff_datetime)
    data.lpep_pickup_datetime = pd.to_datetime(data.lpep_pickup_datetime)

    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data.duration = data.duration.apply(lambda td: td.total_seconds() / 60)
    data = data[(data.duration >= 1) & (data.duration <= 60)]
    
    data['PULocationID'].astype(str, copy=False)
    data['DOLocationID'].astype(str, copy=False)
    
    return data

In [ ]:
num_features = ['trip_distance', 'extra', 'fare_amount']
cat_features = ['PULocationID', 'DOLocationID']

In [ ]:
X_train = process_dataframe(train_raw_data)[num_features + cat_features]
X_val = process_dataframe(val_raw_data)[num_features + cat_features] 

y_train = process_dataframe(train_raw_data)['duration']
y_val = process_dataframe(val_raw_data)['duration'] 

In [ ]:
X_val.isnull().sum()

## Simple Experimnet 

In [ ]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

## MLflow tracking

In [ ]:
import mlflow

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("workspace", "in_class")
    mlflow.log_param("train_data_name", "green_tripdata_2021-01.parquet")
    mlflow.log_param("validation_data_name", "green_tripdata_2021-02.parquet")
    
    alpha = 0.99
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)

## Hyperparameters Optimization

In [ ]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
validation = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        num_boost_round = 500
        early_stopping_rounds = 50
        
        mlflow.log_params(params)
        mlflow.log_param('num_boost_round', num_boost_round)
        mlflow.log_param('early_stopping_rounds', early_stopping_rounds)
        mlflow.log_param('train_data_name', 'green_tripdata_2021-01.parquet')
        mlflow.log_param('validation_data_name', 'green_tripdata_2021-02.parquet')
        mlflow.set_tag('model', 'xgboost')

        booster = xgb.train(
            params = params,
            dtrain = train,
            evals = [(validation, "validation")],
            num_boost_round = num_boost_round,
            early_stopping_rounds = early_stopping_rounds
        )
        
        y_pred = booster.predict(validation)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
        return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
grid_search = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child': hp.loguniform('min_child', -1, 3),
    'seed': 111,
    'objective': 'reg:linear'
}

In [ ]:
best_model = fmin(
    fn=objective,
    space=grid_search,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)

## Train the Best Model

In [ ]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("test")

In [ ]:
#We took the best params from the MLflow interface and copien them here

best_params = {
     
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params = best_params,
    dtrain = train,
    evals = [(validation, "validation")],
    num_boost_round = 500,
    early_stopping_rounds = 50,
)

In [ ]:
key="???"
if best_params.get(key):
    print(best_params.get(key))
else:
    print("no value")

In [ ]:
pd.DataFrame.from_dict([best_params])

In [ ]:
y_pred = booster.predict(validation)

rmse = mean_squared_error(y_val, y_pred, squared=False)
rmse

## Model Logging 

In [ ]:
with open('models/moodel.bin', 'wb') as f_out:
    pickle.dump(booster, f_out)

In [ ]:
with open('preprocessing/process_dataframe.bin', 'wb') as f_out:
    pickle.dump(process_dataframe, f_out)

In [ ]:
mlflow.set_experiment("test")
with mlflow.start_run():
    best_params = {

    }
    
    mlflow.log_params(best_params)
    mlflow.log_param('train_data_name', 'green_tripdata_2022-01.parquet')
    mlflow.log_param('validation_data_name', 'green_tripdata_2022-02.parquet')
    mlflow.set_tag('model', 'xgboost')
    
    booster = xgb.train(
    params = best_params,
    dtrain = train,
    evals = [(validation, "validation")],
    num_boost_round = 500,
    early_stopping_rounds = 50,
    )
    
    mlflow.xgboost.log_model(booster, artifact_path='mlflow_models')
    mlflow.log_artifact('preprocessing/process_dataframe.bin', artifact_path='preprocessing')
    

## Load Model

In [ ]:
logged_model = 'runs:???'
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
type(loaded_model)

In [ ]:
y_preds = loaded_model.predict(X_val)

In [ ]:
mean_squared_error(y_preds, y_val, squared=False)

In [ ]:
y_preds

In [ ]:
print(loaded_model.metadata.get_model_info())

## Sklearn Models

In [ ]:
mlflow.sklearn.autolog()

for algorithm in (LinearSVR, RandomForestRegressor, GradientBoostingRegressor):
    with mlflow.start_run():
        mlflow.log_param('train_data_name', 'green_tripdata_2022-01.parquet')
        mlflow.log_param('validation_data_name', 'green_tripdata_2022-02.parquet')
        mlflow.log_artifact('preprocessing/process_dataframe.bin', artifact_path='preprocessing')
        model = algorithm()
        model.fit(X_train, y_train)
        
        preds = model.predict(X_val)
        rmse = mean_squared_error(preds, y_val, squared=False)
        mlflow.log_metric("rmse", rmse)
        

## MLflow Client

In [ ]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [ ]:
MLFLOW_URI = "sqlite:///mlflow.db"

In [ ]:
client = MlflowClient(MLFLOW_URI)

In [ ]:
client.list_experiments()

In [ ]:
client.create_experiment(name = 'new-experimet')

In [ ]:
client.list_experiments()

In [ ]:
runs = client.search_runs(
    experiment_ids='1',
    run_view_type=ViewType.ACTIVE_ONLY,
    filter_string='metrics.rmse < 7',
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [ ]:
for run in runs:
    print(f"run_id:{run.info.run_id}, metrics:{run.data.metrics['rmse']}")